In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

data_csv = pd.read_csv('/kaggle/input/fsc22-dataset/Metadata-20220916T202011Z-001/Metadata/Metadata V1.0 FSC22.csv')




In [ ]:
data_csv


We have 2025 data sets as our audio data sets.

to Augment the data set and widen it we pitch 2 steps up and down in each audio file

In [ ]:
import librosa
import soundfile as sf
from tqdm import tqdm
import os

input_dir = "/kaggle/input/fsc22-dataset/Audio Wise V1.0-20220916T202003Z-001/Audio Wise V1.0"

output_dir = "/kaggle/working/FSC22_augmented"
os.makedirs(output_dir, exist_ok = True)


PITCH_STEPS = [2,-2]

for file in tqdm(os.listdir(input_dir)):
    if file.endswith(".wav"):
        file_path = os.path.join(input_dir, file)

        y, sr = librosa.load(file_path, sr=None) #loading the audio

        sf.write(os.path.join(output_dir, file),y,sr)

        for step in PITCH_STEPS:
            y_shifted = librosa.effects.pitch_shift(y,sr= sr, n_steps=step)
            base, ext = os.path.splitext(file)
            new_filename=f"{base}_pitch{step}{ext}"
            sf.write(os.path.join(output_dir, new_filename), y_shifted, sr)

print("Augmentation Completed!")




In [ ]:
filename_to_label= {}

for i, row in data_csv.iterrows():
    filename = row["Daraset File name"]
    class_id = row["Class ID"]
    filename_to_label[filename] = class_id

def get_orginal_filename(filename):
    base, ext = os.path.splitext(filename)
    if "_pitch" in base:
        base = base.split("_pitch")[0]
    return base + ext
    

Since we have augmented the data set now we are trying to do feature extraction.

for ausio signal processing, we use MFCC and melspectrogram methods using librosa library

In [ ]:

auto_dir = "/kaggle/working/FSC22_augmented"

#Feature Extraction Parameters

SR = 22050 #sampling rate
N_FFT = 2048  #number of samples per FFT window
HOP_LENGTH = 512 #
N_MELS = 128 #number of mel frequency bands. 128 is ide
N_MFCC = 13 #number of MFCC coefficients

def extract_mel_spectrogram(file_path):
    y, sr = librosa.load(file_path, sr=SR) # Y is the 1D waveform array
    mel_spec = librosa.feature.melspectrogram( y=y, sr=sr, n_fft=N_FFT, hop_length=HOP_LENGTH, n_mel=N_MELS)
    mel_specs_db = librosa.power_to_db(mel_spec. ref=np.max)

    return mel_spec_db

def extract_mfcc(file_path):
    y, sr = librosa.load(file_path, sr=SR)
    mfcc = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=N_MFCC,n_fft=N_FFT,hop_length=HOP_LENGTH)

    #MFCC algorthm works as.. STFT ->MEL filterbank(triangular) -> log -> DCT
    return mfcc



#Looping 


mel_features = []
mfcc_features = []
file_names = []

for file in tqdm(os.listdir(audio_dir)):
    if file.endswith(".wav"):

        path = os.path.join(auto_dir, file):

        mel =extract_mel_spectrogram(path)
        mfcc = extract_mfcc(path)

        mel_features.append(mel)
        mfcc_features.append(mfcc)
        file_names.append(file)

        mel_features = np.array(mel_fetures, dtype = object)
        mfcc_features = np.array(mfcc_features, dtype = object)


        

In [ ]:
import os
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
fromtqdm import tqdm
import librosa

x = []
y = []

#preparing 1D feature 

for mfcc, fname in zip(mfcc_features, file_names):
    orginal_fname = get_orginal_filename(fname)
    label = filename_to_label[orginal_fname]

    mfcc_mean = np.mean(mfcc,axis =1)
    x.append(mfcc_mean)
    y.append(label)
    

from sklearn.preprocessing import LabelEncoder

le  = LabelEncoder
y_encoded = le.fit_transform(y)




In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split( X, y_encoded, test_size=0.2, random_state=42, stratify = y)

import xgboost as xgb

dtrain =xgb.DMatrix( X_train, label = y_train)
dval = xgb.Dmatrix(X_val, label=y_val)

params = {
    "objective": "multi:softmax",
    "num_class" : 27,
    "eval_metric":"merror"
    "subsample":1,
    "min_child_weight":1,
    "max_depth":6,
    "learning_rate":0.3
}

num_rounds = 100

model = xgb.train(
    params,
    dtrain,
    num_boost_roubd = num_rounds,
    evals=[(dval, "validation")])
)

preds = model.predict(dval)
accuracy = (preds==y_val).mean()
print("validation _accuracy:", accuracy)